### Part 1

In [64]:
from collections import deque
def graph_maker(grid, start, endrow=None): #assumes final point only one in its row
    if endrow is None:
        endrow = len(grid)-1
    
    #from start to 1st bifurcation
    neighbors = {tuple(map(sum,zip(start,(1,0))))} #at start, single neighbor always down
    current = start
    edge_weight = 0
    while len(neighbors)==1:
        edge_weight +=1
        previous = current
        current = neighbors.pop() #single neighbor
        neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        neighbors.remove(previous)
        neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
    root = current
    queue = deque({root})  #begin queue with first bifurcation
    pre_root_dist = edge_weight #start point not in graph, distance added to every path anyway, 
    
    #main cycle
    graph = {}
    while queue!=deque([]):
        node = queue.pop()
        graph.update({node: {}})
        edge_weight = 0
        current = node
        followings = set()
        forbids = {(0,1):"#<", (1,0):"#", (0,-1):"#>", (-1,0):"#v"} #dict of forbiddens; there should be a "^", but ^ never on the maps
        for delta in forbids:
            the_next = tuple(map(sum,zip(current,delta)))
            if grid[the_next[0]][the_next[1]] not in forbids[delta]:
                followings.add(the_next)
        for current in followings: #2 or 1 paths to go, depending on junction kind
            delta = {">": (0,1), "v": (1,0), "<":(0,-1)}[grid[current[0]][current[1]]] #there should be a "^", but ^ never on the maps
            neighbors = {tuple(map(sum,zip(current,delta)))} #always a pointed spot, always one path to go
            edge_weight = 1
            while len(neighbors)==1:
                edge_weight +=1
                previous = current
                current = neighbors.pop() #single neighbor
                if current[0]==endrow: #final point
                    post_leaf_dist = edge_weight
                    del graph[node]
                    leaf = node
                    break
                neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
                neighbors.remove(previous)
                neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
            else: #final point not in graph
                graph[node].update({current: edge_weight})
                queue.appendleft(current) #add point to queue, except final one
    return graph, root, pre_root_dist, leaf, post_leaf_dist
def tree_maker(graph, root, leaf):
    n_copies = {pt:-1 for pt in graph} #how many copies of pt have entered the tree, for proper index numbering, -1 to correct off-by-one weirdness
    n_copies.update({leaf:-1})
    tree = {}#node:parent

    #root loop separately, simplifies a bit
    children = set(graph[root].keys())
    for pt in children:
        n_copies[pt]=0
    children = {(pt, n_copies[pt]) for pt in children}
    tree.update({child: (root,0) for child in children})
    queue = deque(children) #better queueing with deque

    #main loop
    while queue!=deque([]):
        node = queue.pop()
        if node[0]==leaf:
            continue
        # no worry with ascendants, directed graph takes care of it
        children = set(graph[node[0]].keys()) #graph[node[0]] #set of 2-tuples
        #no worry with non-finish leaves, all leaves are finish leaves
        for pt in children:
            n_copies[pt]+=1
        children = {(pt, n_copies[pt]) for pt in children}
        tree.update({child: node for child in children})
        queue.extend(children)

    return tree
def max_length(graph, root, pre_root_dist, leaf, post_leaf_dist, tree):
    fin_leaves = {node for node in tree if node[0]==leaf}
    max_len = 0
    for leaf in fin_leaves:
        length = pre_root_dist + post_leaf_dist
        parent = tree[leaf]
        length += graph[parent[0]][leaf[0]]
        while parent != (root,0):
            current = parent
            parent = tree[current]
            length += graph[parent[0]][current[0]]
        max_len = length if length > max_len else max_len
    return max_len

with open("input23.txt", "r") as f:
    grid = f.read().split("\n")

start = (0,1)
# start, end = (0,1), (len(grid)-1,len(grid[0])-2) #valid for example and actual input

graph, root, pre_root_dist, leaf, post_leaf_dist = graph_maker(grid, start)
tree = tree_maker(graph, root, leaf)
print("No of nodes:", len(graph), "\b, size of tree (no of links):", len(tree), "\b, max length is:")
max_length(graph, root, pre_root_dist, leaf, post_leaf_dist, tree)

No of nodes: 33, size of tree (no of links): 920, max length is:


2190

### Part 2

In [115]:
#my solution, almost as nice (in time) as the imported one: 26.5s
from collections import deque
def graph_maker(grid, start, end):
    #from start to 1st bifurcation
    neighbors = {tuple(map(sum,zip(start,(1,0))))} #at start, single neighbor always down
    current = start
    edge_weight = 0
    while len(neighbors)==1:
        edge_weight +=1
        previous = current
        current = neighbors.pop() #single neighbor
        neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        neighbors.remove(previous)
        neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
    root = current
    queue = deque({root})  #begin queue with first bifurcation
    visited = {previous} #set of visited points neighbors to bifurcations
    pre_root_dist = edge_weight #start point not in graph, distance added to every path anyway

    #from end to finish point, find fin_leaf
    neighbors = {tuple(map(sum,zip(end,(-1,0))))} #at the end, single neighbor always up
    current = end
    edge_weight = 0
    while len(neighbors)==1:
        edge_weight +=1
        previous = current
        current = neighbors.pop() #single neighbor
        neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        neighbors.remove(previous)
        neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
    fin_leaf = current
    visited.add(previous) #set of visited points neighbors to bifurcations
    post_gr_fin_dist = edge_weight

    #main cycle
    graph = {root: {}}
    while queue!=deque([]):
        node = queue.pop()

        edge_weight = 0
        current = node
        
        followings = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        followings = followings - visited
        followings = {f for f in followings if grid[f[0]][f[1]]!="#"}

        for current in followings: #1, 2, or 3 paths to go, depending on junction kind
            neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
            neighbors.remove(node)
            neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}

            edge_weight = 1
            while len(neighbors)==1:
                edge_weight +=1
                previous = current
                current = neighbors.pop() #single neighbor
                neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
                neighbors.remove(previous)
                neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
            graph[node].update({current: edge_weight})
            if current not in graph:
                graph.update({current: {node: edge_weight}})
            else:
                graph[current].update({node: edge_weight})
            visited.add(previous)
            if current != fin_leaf:
                queue.appendleft(current) #add point to queue, except final one

    return graph, root, pre_root_dist, fin_leaf, post_gr_fin_dist
#the obvious improvement would be to use all_paths as a set, not list. But it contains history, which is a set (or list), which is unhashable (both are)
#you get the "unhashable type: 'set'"" error, even with them in a tuple. One advantage of the function is exactly allowing to pass a set
def list_traversal(graph, root, init_dist=0):
    all_paths = [(root, init_dist, set())] #deque actually a bit slower
    best = 0
    while all_paths != []:    
        node, dist, history = all_paths.pop()
        history = history.union({node})
        for adj, delta in graph[node].items():
            if adj not in history:
                if adj == fin_leaf:
                    best = max(dist+delta, best)
                else:
                    all_paths.append((adj, dist+delta, history))
    return best

with open("input23.txt", "r") as f:
    grid = f.read().split("\n")
start, end = (0,1), (len(grid)-1,len(grid[0])-2) #valid for example and actual input
graph, root, pre_root_dist, fin_leaf, post_gr_fin_dist = graph_maker(grid, start, end)
best = list_traversal(graph, root, init_dist=pre_root_dist+post_gr_fin_dist)
print("No of nodes:", len(graph), "\b, max length:", best)

No of nodes: 34, max length: 6258


In [102]:
#Using Sven's function
from collections import deque
def graph_maker(grid, start, end):
    #from start to 1st bifurcation
    neighbors = {tuple(map(sum,zip(start,(1,0))))} #at start, single neighbor always down
    current = start
    edge_weight = 0
    while len(neighbors)==1:
        edge_weight +=1
        previous = current
        current = neighbors.pop() #single neighbor
        neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        neighbors.remove(previous)
        neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
    root = current
    queue = deque({root})  #begin queue with first bifurcation
    visited = {previous} #set of visited points neighbors to bifurcations
    pre_root_dist = edge_weight #start point not in graph, distance added to every path anyway

    #from end to finish point, find fin_leaf
    neighbors = {tuple(map(sum,zip(end,(-1,0))))} #at the end, single neighbor always up
    current = end
    edge_weight = 0
    while len(neighbors)==1:
        edge_weight +=1
        previous = current
        current = neighbors.pop() #single neighbor
        neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        neighbors.remove(previous)
        neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
    fin_leaf = current
    visited.add(previous) #set of visited points neighbors to bifurcations
    post_gr_fin_dist = edge_weight

    #main cycle
    graph = {root: {}}
    while queue!=deque([]):
        node = queue.pop()

        edge_weight = 0
        current = node
        
        followings = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        followings = followings - visited
        followings = {f for f in followings if grid[f[0]][f[1]]!="#"}

        for current in followings: #1, 2, or 3 paths to go, depending on junction kind
            neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
            neighbors.remove(node)
            neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}

            edge_weight = 1
            while len(neighbors)==1:
                edge_weight +=1
                previous = current
                current = neighbors.pop() #single neighbor
                neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
                neighbors.remove(previous)
                neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
            graph[node].update({current: edge_weight})
            if current not in graph:
                graph.update({current: {node: edge_weight}})
            else:
                graph[current].update({node: edge_weight})
            visited.add(previous)
            if current != fin_leaf:
                queue.appendleft(current) #add point to queue, except final one

    return graph, root, pre_root_dist, fin_leaf, post_gr_fin_dist

#not path to finish leaf, but tree to finish leaf!
def path_to_fin_leaf(node, history):
    from collections import deque
    nodes_to_check = deque(set(graph[node].keys()) - history)
    path, nodes_visited = {node}, {node}
    while nodes_to_check:      
        curr_node = nodes_to_check.pop()
        if curr_node == fin_leaf:
            path.add(curr_node)
            return (path)
        nodes_visited.add(curr_node)
        new_nodes = set(graph[curr_node].keys()) - (history.union(nodes_visited))
        if new_nodes:
            nodes_to_check.extend(new_nodes)
            path.add(curr_node)      
    return None

#the obvious improvement would be to use all_paths as a set, not list. But it contains history, which is a set (or list), which is unhashable (both are)
#you get the "unhashable type: 'set'"" error, even with them in a tuple. One advantage of the function is exactly allowing to pass a set
def list_traversal(graph, root, init_dist=0):
    all_paths = [(root, init_dist, set())] #deque actually a bit slower
    best = 0
    while all_paths != []:    
        node, dist, history = all_paths.pop()
        history = history.union({node})
        for adj, delta in graph[node].items():
            if adj in history:
                continue
            if adj == fin_leaf:
                best = max(dist+delta, best)
                continue
            if quickpaths[adj].intersection(history)!=set():
                new_path = path_to_fin_leaf(adj, history)
                if new_path is None:
                    continue
                quickpaths[adj] = new_path
            all_paths.append((adj, dist+delta, history))
    return best

with open("input23.txt", "r") as f:
    grid = f.read().split("\n")
start, end = (0,1), (len(grid)-1,len(grid[0])-2) #valid for example and actual input
graph, root, pre_root_dist, fin_leaf, post_gr_fin_dist = graph_maker(grid, start, end)
quickpaths = {node: path_to_fin_leaf(node, set()) for node in graph.keys()}
best = list_traversal(graph, root, init_dist=pre_root_dist+post_gr_fin_dist)
print("No of nodes:", len(graph), "\b, max length:", best)

No of nodes: 34, max length: 6258


In [109]:
#variant from discussion with Sven. A bit slower with 35s (against 29 above at the same time)
from collections import deque
def graph_maker(grid, start, end):
    #from start to 1st bifurcation
    neighbors = {tuple(map(sum,zip(start,(1,0))))} #at start, single neighbor always down
    current = start
    edge_weight = 0
    while len(neighbors)==1:
        edge_weight +=1
        previous = current
        current = neighbors.pop() #single neighbor
        neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        neighbors.remove(previous)
        neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
    root = current
    queue = deque({root})  #begin queue with first bifurcation
    visited = {previous} #set of visited points neighbors to bifurcations
    pre_root_dist = edge_weight #start point not in graph, distance added to every path anyway

    #from end to finish point, find fin_leaf
    neighbors = {tuple(map(sum,zip(end,(-1,0))))} #at the end, single neighbor always up
    current = end
    edge_weight = 0
    while len(neighbors)==1:
        edge_weight +=1
        previous = current
        current = neighbors.pop() #single neighbor
        neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        neighbors.remove(previous)
        neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
    fin_leaf = current
    visited.add(previous) #set of visited points neighbors to bifurcations
    post_gr_fin_dist = edge_weight

    #main cycle
    graph = {root: {}}
    while queue!=deque([]):
        node = queue.pop()

        edge_weight = 0
        current = node
        
        followings = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        followings = followings - visited
        followings = {f for f in followings if grid[f[0]][f[1]]!="#"}

        for current in followings: #1, 2, or 3 paths to go, depending on junction kind
            neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
            neighbors.remove(node)
            neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}

            edge_weight = 1
            while len(neighbors)==1:
                edge_weight +=1
                previous = current
                current = neighbors.pop() #single neighbor
                neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
                neighbors.remove(previous)
                neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
            graph[node].update({current: edge_weight})
            if current not in graph:
                graph.update({current: {node: edge_weight}})
            else:
                graph[current].update({node: edge_weight})
            visited.add(previous)
            if current != fin_leaf:
                queue.appendleft(current) #add point to queue, except final one

    return graph, root, pre_root_dist, fin_leaf, post_gr_fin_dist
#the obvious improvement would be to use all_paths as a set, not list. But it contains history, which is a set (or list), which is unhashable (both are)
#you get the "unhashable type: 'set'"" error, even with them in a tuple. One advantage of the function is exactly allowing to pass a set
from collections import deque
def all_paths_to_finish(fin_leaf):
    short_paths = {node: set() for node in graph}
    short_paths[fin_leaf] = {fin_leaf}
    queue = deque({fin_leaf})
    while set() in short_paths.values():
        curr_node = queue.pop()
        this_path = short_paths[curr_node].union({curr_node})
        for adj in graph[curr_node].keys() - this_path:
            if short_paths[adj]==set():
                short_paths[adj] = this_path
                queue.appendleft(adj)
    del short_paths[fin_leaf]
    return short_paths

def some_paths_to_finish(node, fin_leaf, history):
    new_short_paths = {fin_leaf: {fin_leaf}}
    queue = deque({fin_leaf})
    while queue!=deque({}) and node not in new_short_paths:
        curr_node = queue.pop()
        this_path = new_short_paths[curr_node].union({curr_node})
        for adj in graph[curr_node].keys() - history.union(this_path):
            if adj not in new_short_paths:
                new_short_paths.update({adj: this_path})
                queue.appendleft(adj)
    del new_short_paths[fin_leaf]
    return new_short_paths

def list_traversal_cached(graph, root, fin_leaf, init_dist=0):
    all_paths = [(root, init_dist, set())]
    best = 0
    while all_paths != []:    
        node, dist, history = all_paths.pop()
        history = history.union({node})
        for adj, delta in graph[node].items():
            if adj in history:
                continue
            if adj == fin_leaf:
                best = max(dist+delta, best)
                continue
            if shortpaths[adj].intersection(history) != set():
                shortpaths[adj] = set()
                new_shortpaths = some_paths_to_finish(adj, fin_leaf, history)
                shortpaths.update(new_shortpaths)
                if adj not in new_shortpaths:
                    continue
            all_paths.append((adj, dist+delta, history))
    return best

with open("input23.txt", "r") as f:
    grid = f.read().split("\n")
start, end = (0,1), (len(grid)-1,len(grid[0])-2) #valid for example and actual input
graph, root, pre_root_dist, fin_leaf, post_gr_fin_dist = graph_maker(grid, start, end)
shortpaths = all_paths_to_finish(fin_leaf)
best = list_traversal_cached(graph, root, fin_leaf, init_dist=pre_root_dist+post_gr_fin_dist)
print("No of nodes:", len(graph), "\b, max length:", best)

No of nodes: 34, max length: 6258


In [106]:
#the nice solution, imported and with my enhancements to reach 23.5s
from collections import deque
def graph_maker(grid, start, end): #assumes final point only one in its row
    #from start to 1st bifurcation
    neighbors = {tuple(map(sum,zip(start,(1,0))))} #at start, single neighbor always down
    current = start
    edge_weight = 0
    while len(neighbors)==1:
        edge_weight +=1
        previous = current
        current = neighbors.pop() #single neighbor
        neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        neighbors.remove(previous)
        neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
    root = current
    queue = deque({root})  #begin queue with first bifurcation
    visited = {previous} #set of visited points neighbors to bifurcations
    pre_root_dist = edge_weight #start point not in graph, distance added to every path anyway

    #from end to finish point, find fin_leaf
    neighbors = {tuple(map(sum,zip(end,(-1,0))))} #at the end, single neighbor always up
    current = end
    edge_weight = 0
    while len(neighbors)==1:
        edge_weight +=1
        previous = current
        current = neighbors.pop() #single neighbor
        neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        neighbors.remove(previous)
        neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
    fin_leaf = current
    visited.add(previous) #set of visited points neighbors to bifurcations
    post_gr_fin_dist = edge_weight

    #main cycle
    graph = {root: {}}
    while queue!=deque([]):
        node = queue.pop()

        edge_weight = 0
        current = node
        
        followings = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        followings = followings - visited
        followings = {f for f in followings if grid[f[0]][f[1]]!="#"}

        for current in followings: #1, 2, or 3 paths to go, depending on junction kind
            neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
            neighbors.remove(node)
            neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}

            edge_weight = 1
            while len(neighbors)==1:
                edge_weight +=1
                previous = current
                current = neighbors.pop() #single neighbor
                neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
                neighbors.remove(previous)
                neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
            graph[node].update({current: edge_weight})
            if current not in graph:
                graph.update({current: {node: edge_weight}})
            else:
                graph[current].update({node: edge_weight})
            visited.add(previous)
            if current != fin_leaf:
                queue.appendleft(current) #add point to queue, except final one

    return graph, root, pre_root_dist, fin_leaf, post_gr_fin_dist
#there could've been other choices of global x local (passed-on) variables
#making best_sofar an argument of the function gains more than 20s (time cut in half, 44s to 23s), but extending it to graph, fin_leaf doesn't;
#I guess it makes sense to add changing variables as arguments, let the fixed ones as globals
def traversal(node, pathset, dist, best_dist):#, graph, fin_leaf):
    if node == fin_leaf:
        return max(best_dist, dist)
    for nb in set(graph[node].keys()):
        if nb not in pathset:
            pathset.add(nb)
            best_dist = traversal(nb, pathset, dist+graph[node][nb], best_dist)#, graph, fin_leaf)
            pathset.remove(nb)
    return best_dist

with open("input23.txt", "r") as f:
    grid = f.read().split("\n")

start, end = (0,1), (len(grid)-1,len(grid[0])-2) #valid for example and actual input
graph, root, pre_root_dist, fin_leaf, post_gr_fin_dist = graph_maker(grid, start, end)
max_len = traversal(root, {root}, pre_root_dist + post_gr_fin_dist, 0)
print("No of nodes:", len(graph), "\b, max length:", max_len)

No of nodes: 34, max length: 6258


In [116]:
#the nice solution, adding path check
from collections import deque
def graph_maker(grid, start, end): #assumes final point only one in its row
    #from start to 1st bifurcation
    neighbors = {tuple(map(sum,zip(start,(1,0))))} #at start, single neighbor always down
    current = start
    edge_weight = 0
    while len(neighbors)==1:
        edge_weight +=1
        previous = current
        current = neighbors.pop() #single neighbor
        neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        neighbors.remove(previous)
        neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
    root = current
    queue = deque({root})  #begin queue with first bifurcation
    visited = {previous} #set of visited points neighbors to bifurcations
    pre_root_dist = edge_weight #start point not in graph, distance added to every path anyway

    #from end to finish point, find fin_leaf
    neighbors = {tuple(map(sum,zip(end,(-1,0))))} #at the end, single neighbor always up
    current = end
    edge_weight = 0
    while len(neighbors)==1:
        edge_weight +=1
        previous = current
        current = neighbors.pop() #single neighbor
        neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        neighbors.remove(previous)
        neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
    fin_leaf = current
    visited.add(previous) #set of visited points neighbors to bifurcations
    post_gr_fin_dist = edge_weight

    #main cycle
    graph = {root: {}}
    while queue!=deque([]):
        node = queue.pop()

        edge_weight = 0
        current = node
        
        followings = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        followings = followings - visited
        followings = {f for f in followings if grid[f[0]][f[1]]!="#"}

        for current in followings: #1, 2, or 3 paths to go, depending on junction kind
            neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
            neighbors.remove(node)
            neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}

            edge_weight = 1
            while len(neighbors)==1:
                edge_weight +=1
                previous = current
                current = neighbors.pop() #single neighbor
                neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
                neighbors.remove(previous)
                neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
            graph[node].update({current: edge_weight})
            if current not in graph:
                graph.update({current: {node: edge_weight}})
            else:
                graph[current].update({node: edge_weight})
            visited.add(previous)
            if current != fin_leaf:
                queue.appendleft(current) #add point to queue, except final one

    return graph, root, pre_root_dist, fin_leaf, post_gr_fin_dist

def all_paths_to_finish(fin_leaf):
    short_paths = {node: set() for node in graph}
    short_paths[fin_leaf] = {fin_leaf}
    queue = deque({fin_leaf})
    while set() in short_paths.values():
        curr_node = queue.pop()
        this_path = short_paths[curr_node].union({curr_node})
        for adj in graph[curr_node].keys() - this_path:
            if short_paths[adj]==set():
                short_paths[adj] = this_path
                queue.appendleft(adj)
    # del short_paths[fin_leaf]
    return short_paths

def some_paths_to_finish(node, fin_leaf, history):
    new_short_paths = {fin_leaf: {fin_leaf}}
    queue = deque({fin_leaf})
    while queue!=deque({}) and node not in new_short_paths:
        curr_node = queue.pop()
        this_path = new_short_paths[curr_node].union({curr_node})
        for adj in graph[curr_node].keys() - history.union(this_path):
            if adj not in new_short_paths:
                new_short_paths.update({adj: this_path})
                queue.appendleft(adj)
    del new_short_paths[fin_leaf]
    return new_short_paths

#there could've been other choices of global x local (passed-on) variables
#making best_sofar an argument of the function gains more than 20s (time cut in half, 44s to 23s), but extending it to graph, fin_leaf doesn't;
#I guess it makes sense to add changing variables as arguments, let the fixed ones as globals
def traversal(node, pathset, dist, best_dist):#, graph, fin_leaf):
    if node == fin_leaf:
        return max(best_dist, dist)
    for nb in set(graph[node].keys()):
        if nb not in pathset:
            if quickpaths[nb].intersection(pathset)!=set():
                new_quickpath = path_to_fin_leaf(nb, pathset)
                if new_quickpath is None:
                    continue
                # quickpaths[nb] = set()
                # new_quickpaths = some_paths_to_finish(nb, fin_leaf, pathset)
                # quickpaths.update(new_quickpaths)
                # if nb not in new_quickpaths:
                #     continue
            pathset.add(nb)
            best_dist = traversal(nb, pathset, dist+graph[node][nb], best_dist)#, graph, fin_leaf)
            pathset.remove(nb)
    return best_dist

with open("input23.txt", "r") as f:
    grid = f.read().split("\n")

start, end = (0,1), (len(grid)-1,len(grid[0])-2) #valid for example and actual input
graph, root, pre_root_dist, fin_leaf, post_gr_fin_dist = graph_maker(grid, start, end)
quickpaths = all_paths_to_finish(fin_leaf)
max_len = traversal(root, {root}, pre_root_dist + post_gr_fin_dist, 0)
print("No of nodes:", len(graph), "\b, max length:", max_len)

No of nodes: 34, max length: 6258


In [ ]:
from collections import deque
#Additional information: the actual path that is longest (takes little less than 1min with my method, imported takes longer)
def graph_maker(grid, start, end): #assumes final point only one in its row
    #from start to 1st bifurcation
    neighbors = {tuple(map(sum,zip(start,(1,0))))} #at start, single neighbor always down
    current = start
    edge_weight = 0
    while len(neighbors)==1:
        edge_weight +=1
        previous = current
        current = neighbors.pop() #single neighbor
        neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        neighbors.remove(previous)
        neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
    root = current
    queue = deque({root})  #begin queue with first bifurcation
    visited = {previous} #set of visited points neighbors to bifurcations
    pre_root_dist = edge_weight #start point not in graph, distance added to every path anyway

    #from end to finish point, find fin_leaf
    neighbors = {tuple(map(sum,zip(end,(-1,0))))} #at the end, single neighbor always up
    current = end
    edge_weight = 0
    while len(neighbors)==1:
        edge_weight +=1
        previous = current
        current = neighbors.pop() #single neighbor
        neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        neighbors.remove(previous)
        neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
    fin_leaf = current
    visited.add(previous) #set of visited points neighbors to bifurcations
    post_gr_fin_dist = edge_weight

    #main cycle
    graph = {root: {}}
    while queue!=deque([]):
        node = queue.pop()

        edge_weight = 0
        current = node
        
        followings = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        followings = followings - visited
        followings = {f for f in followings if grid[f[0]][f[1]]!="#"}

        for current in followings: #1, 2, or 3 paths to go, depending on junction kind
            neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
            neighbors.remove(node)
            neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}

            edge_weight = 1
            while len(neighbors)==1:
                edge_weight +=1
                previous = current
                current = neighbors.pop() #single neighbor
                neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
                neighbors.remove(previous)
                neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
            graph[node].update({current: edge_weight})
            if current not in graph:
                graph.update({current: {node: edge_weight}})
            else:
                graph[current].update({node: edge_weight})
            visited.add(previous)
            if current != fin_leaf:
                queue.appendleft(current) #add point to queue, except final one

    return graph, root, pre_root_dist, fin_leaf, post_gr_fin_dist

#imported
def traversal(node, path, dist, best_sofar, bestpath):#, graph, fin_leaf):
    if node == fin_leaf:
        if dist > best_sofar:
            bestpath = path.copy()
            best_sofar = dist
        return best_sofar, bestpath
    for nb in set(graph[node].keys()):
        if nb not in path:
            path.append(nb)
            best_sofar, bestpath = traversal(nb, path, dist+graph[node][nb], best_sofar, bestpath)#, graph, fin_leaf)
            path.remove(nb)
    return best_sofar, bestpath

#mine
def list_traversal(root,graph, init_dist=0): #mine actually faster for keeping path info
    all_paths = [(root, init_dist, [])]
    best, bestpath = 0, []
    while all_paths != []:    
        node, dist, history = all_paths.pop()
        history = history+[node]
        for adj, delta in graph[node].items():
            if adj not in history:
                if adj == fin_leaf:
                    if dist+delta > best:
                        best = dist+delta
                        bestpath = history+[adj]
                else:
                    all_paths.append((adj, dist+delta, history))
    return best, bestpath

with open("input23.txt", "r") as f:
    grid = f.read().split("\n")

start, end = (0,1), (len(grid)-1,len(grid[0])-2)
graph, root, pre_root_dist, fin_leaf, post_gr_fin_dist = graph_maker(grid, start, end)
# traversal(root, [root], pre_root_dist + post_gr_fin_dist, 0, [])
list_traversal(root, graph, init_dist=pre_root_dist + post_gr_fin_dist)

In [ ]:
from collections import deque
#works, but it's SLOW, close to 5min total runtime: there's no need to build a full tree
def graph_maker(grid, start, end): #assumes final point only one in its row
    #from start to 1st bifurcation
    neighbors = {tuple(map(sum,zip(start,(1,0))))} #at start, single neighbor always down
    current = start
    edge_weight = 0
    while len(neighbors)==1:
        edge_weight +=1
        previous = current
        current = neighbors.pop() #single neighbor
        neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        neighbors.remove(previous)
        neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
    root = current
    queue = deque({root})  #begin queue with first bifurcation
    visited = {previous} #set of visited points neighbors to bifurcations
    pre_root_dist = edge_weight #start point not in graph, distance added to every path anyway

    #from end to finish point, find fin_leaf
    neighbors = {tuple(map(sum,zip(end,(-1,0))))} #at the end, single neighbor always up
    current = end
    edge_weight = 0
    while len(neighbors)==1:
        edge_weight +=1
        previous = current
        current = neighbors.pop() #single neighbor
        neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        neighbors.remove(previous)
        neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
    fin_leaf = current
    visited.add(previous) #set of visited points neighbors to bifurcations
    post_gr_fin_dist = edge_weight

    #main cycle
    graph = {root: {}}
    while queue!=deque([]):
        node = queue.pop()

        edge_weight = 0
        current = node
        
        followings = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        followings = followings - visited
        followings = {f for f in followings if grid[f[0]][f[1]]!="#"}

        for current in followings: #1, 2, or 3 paths to go, depending on junction kind
            neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
            neighbors.remove(node)
            neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}

            edge_weight = 1
            while len(neighbors)==1:
                edge_weight +=1
                previous = current
                current = neighbors.pop() #single neighbor
                neighbors = {tuple(map(sum,zip(current,delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
                neighbors.remove(previous)
                neighbors = {n for n in neighbors if grid[n[0]][n[1]]!="#"}
            graph[node].update({current: edge_weight})
            if current not in graph:
                graph.update({current: {node: edge_weight}})
            else:
                graph[current].update({node: edge_weight})
            visited.add(previous)
            if current != fin_leaf:
                queue.appendleft(current) #add point to queue, except final one

    return graph, root, pre_root_dist, fin_leaf, post_gr_fin_dist
def tree_maker(graph, root, fin_leaf):
    n_copies = {pt:-1 for pt in graph} #how many copies of pt have entered the tree, for proper index numbering, -1 to correct off-by-one weirdness
    n_copies.update({fin_leaf:-1})
    tree = {}#node:parent

    #root loop separately, simplifies a bit
    children = set(graph[root].keys())
    for pt in children:
        n_copies[pt]=0
    children = {(pt, n_copies[pt]) for pt in children}
    tree.update({child: (root,0) for child in children})
    queue = deque(children) #better queueing with deque

    #main loop
    while queue!=deque([]):
        node = queue.pop()
        if node[0]==fin_leaf:
            continue
        ascendants = [tree[node]]
        while ascendants[-1] != (root,0):
            ascendants.append(tree[ascendants[-1]])
        children = set(graph[node[0]].keys()) #set of 2-tuples
        children = children - {asc[0] for asc in ascendants}
        if children == set(): #PRUNE node and some ascendants!
            #del more
            del tree[node]
            continue 
        for pt in children:
            n_copies[pt]+=1
        children = {(pt, n_copies[pt]) for pt in children}
        tree.update({child: node for child in children})
        queue.extendleft(children)
        # print("\rlen(tree): "+str(len(tree)), end="       ")
    return tree
def max_length(graph, root, pre_root_dist, leaf, post_leaf_dist, tree):
    fin_leaves = {node for node in tree if node[0]==leaf}
    max_len = 0
    for leaf in fin_leaves:
        length = pre_root_dist + post_leaf_dist
        parent = tree[leaf]
        length += graph[parent[0]][leaf[0]]
        while parent != (root,0):
            current = parent
            parent = tree[current]
            length += graph[parent[0]][current[0]]
        max_len = length if length > max_len else max_len
    return max_len

with open("input23.txt", "r") as f:
    grid = f.read().split("\n")

start, end = (0,1), (len(grid)-1,len(grid[0])-2) #valid for example and actual input
graph, root, pre_root_dist, fin_leaf, post_gr_fin_dist = graph_maker(grid, start, end)
tree = tree_maker(graph, root, fin_leaf)
max_len = max_length(graph, root, pre_root_dist, fin_leaf, post_gr_fin_dist, tree)

print("No of nodes:", len(graph), "\b, size of tree (no of links):", len(tree), "\b, max length:", max_len)

In [ ]:
#alternatives with more info (Part 1)
#max length and also max-length path
def max_length_path(graph, root, pre_root_dist, leaf, post_leaf_dist, tree):
    fin_leaves = {node for node in tree if node[0]==leaf}
    max_len = 0
    for leaf in fin_leaves:
        length = pre_root_dist + post_leaf_dist
        ascendants = [tree[leaf]]
        length += graph[ascendants[-1][0]][leaf[0]]
        while ascendants[-1] != (root,0):
            ascendants.append(tree[ascendants[-1]])
            length += graph[ascendants[-1][0]][ascendants[-2][0]]
        if length > max_len:
            max_len = length
    return max_len, {asc[0] for asc in ascendants[::-1]}

#return all possible path lengths
def all_path_lengths(graph, root, pre_root_dist, leaf, post_leaf_dist, tree):
    fin_leaves = {node for node in tree if node[0]==leaf}
    path_lengths = []
    for leaf in fin_leaves:
        length = pre_root_dist + post_leaf_dist
        ascendants = [tree[leaf]]
        length += graph[ascendants[-1][0]][leaf[0]]
        while ascendants[-1] != (root,0):
            ascendants.append(tree[ascendants[-1]])
            length += graph[ascendants[-1][0]][ascendants[-2][0]]
        path_lengths.append(length)
    return path_lengths#, {asc[0] for asc in ascendants[::-1]}

import matplotlib.pyplot as plt
plt.hist(all_path_lengths(graph, root, pre_root_dist, leaf, post_leaf_dist, tree));
max_length_path(graph, root, pre_root_dist, leaf, post_leaf_dist, tree)

In [ ]:
#on small, free grid to get the ideas going
#this is not super efficient, but notice that the input has few bifurcations, so maybe it's doable like this
#but weights between node and child are needed
from collections import deque

graph = {}
grid_size = 5
for i in range(grid_size):
    for j in range(grid_size):
        neighbors = {tuple(map(sum,zip((i,j),delta))) for delta in {(0,1), (1,0), (0,-1), (-1,0)}}
        neighbors = {n for n in neighbors if n[0] in range(grid_size) and n[1] in range(grid_size)}
        graph.update({(i,j): neighbors})

n_copies = {pt:-1 for pt in graph} #how many copies of pt have entered the tree, for proper index numbering, -1 to correct off-by-one weirdness
start  = (0,0)
finish = (grid_size-1,grid_size-1)

tree = {}#node:parent
root = ((0,0),0)

#root loop separately, simplifies a bit
children = graph[root[0]] #set of 2-tuples
for pt in children:
    n_copies[pt]+=1
children = {(pt, n_copies[pt]) for pt in children}
tree.update({child: root for child in children})
queue = deque(children) #better queueing with deque
#main loop
while queue!=deque([]):
    node = queue.pop()
    if node[0]==finish:
        continue
    ascendants = [tree[node]]
    while ascendants[-1] != root:
        ascendants.append(tree[ascendants[-1]])
    children = graph[node[0]] #set of 2-tuples
    children = {pt for pt in children if pt not in {asc[0] for asc in ascendants}}
    if children == set(): #PRUNE node and some ascendants!
        #del more
        del tree[node]
        continue    
    for pt in children:
        n_copies[pt]+=1
    children = {(pt, n_copies[pt]) for pt in children}
    tree.update({child: node for child in children})
    queue.extend(children)
len(tree)

In [ ]:
fin_leaves = {node for node in tree if node[0]==finish}
all_ascendants = set()
for leaf in fin_leaves:#I assume they're leaves because of the finish condition
    ascendants = [tree[leaf]]
    while ascendants[-1] != root:
        ascendants.append(tree[ascendants[-1]])
    all_ascendants.update(ascendants)
    
tree = {node:tree[node] for node in all_ascendants.union(fin_leaves)-{((0,0),0)}}
len(tree)

In [ ]:
fin_leaves = {node for node in tree if node[0]==finish}
path_lengths = set()
for leaf in fin_leaves:#I assume they're leaves because of the finish condition
    ascendants = [tree[leaf]]
    while ascendants[-1] != root:
        ascendants.append(tree[ascendants[-1]])
    print(len(ascendants))
    path_lengths.add(len(ascendants))
print("max:",max(path_lengths))

Note: pruning turned out not to be useful, because we're not really building the tree

In [ ]:
#pruning: prune out leaves that are not the finish
nonfin_leaves = {node for node in tree if node[0]!=finish and node not in tree.values()}
while nonfin_leaves!=set():
    # for 
    tree = {n:p for n,p in tree.items() if n not in nonfin_leaves}
    nonfin_leaves = {node for node in tree if node[0]!=finish and node not in tree.values()}
len(tree)